## README ❗

Set a manual_seed for reproducibility.

References:

- See [Reproducibility](https://pytorch.org/docs/stable/notes/randomness.html)






### TO-DO ✅


#### FIX 🧯


Quando viene sollevata un'eccezione in DataPreprocessor.__getitem __ viene restituito uno stack di immagini vuote. Aggiungere il relativo ID ad una lista e in fase di training evitare che queste immagini nere vengano incluse nel batch. Magari sostituendo a queste una delle immagini valide presenti nello stesso batch.


## Environment setup 🏛


In [1]:
! pip install python-dotenv
! pip install monai
! pip install shutil
! pip install mlflow --quiet
! pip install pyngrok --quiet
! pip install torchmetrics
! pip install dgl

import os
import sys
from google.colab import drive

drive.mount('/content/drive/')
sys.path.append('/content/drive/MyDrive/Lorusso/BraTS/')


from src.preprocess.image_processing import *
from src.preprocess.nifti_io import *
from src.preprocess.graphgen import *
from src.preprocess.graph_io import *

from mlflow.models.signature import infer_signature
import mlflow
import mlflow.pytorch
from pyngrok import ngrok

import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import concurrent.futures
import tarfile
import nibabel as nib
import glob
import random
import shutil
import time
from distutils.dir_util import copy_tree

from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import SequentialSampler
from torchvision import utils

from monai.networks.nets import AutoEncoder
from monai.losses import DiceCELoss, DiceFocalLoss, DiceLoss, FocalLoss

from dgl import from_networkx as to_dgl_graph


#torch.use_deterministic_algorithms(True, warn_only=True)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

DGL backend not selected or invalid.  Assuming PyTorch for now.


In [2]:
%%bash

if [ ! -f '/content/drive/MyDrive/Lorusso/BraTS/.env' ]; then
    echo "Creating .env file..."
    echo "INPUT_DATA_DIR='/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats'" > '/content/drive/MyDrive/Lorusso/BraTS/.env'
    echo "PROCESSED_DATA_DIR='/content/drive/MyDrive/Lorusso/BraTS/data/processed'" >> '/content/drive/MyDrive/Lorusso/BraTS/.env'
fi

In [3]:
%%bash

if [ ! -f '/content/drive/MyDrive/Lorusso/BraTS/data/raw/BraTS2021_Training_Data.tar'  ]; then
    echo "Downloading BraTS dataset at /content/drive/MyDrive/Lorusso/BraTS/data/raw ..."
    mkdir /root/.kaggle/
    cp '/content/drive/MyDrive/Lorusso/kaggle.json' /root/.kaggle
    chmod 600 '/root/.kaggle/kaggle.json'
    cd '/content/drive/MyDrive/Lorusso/BraTS/data/raw' && kaggle datasets download -d dschettler8845/brats-2021-task1
    ls '/content/drive/MyDrive/Lorusso/BraTS/data/raw'
    unzip '/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats-2021-task1.zip' -d '/content/drive/MyDrive/Lorusso/BraTS/data/raw/'
    rm -r '/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats-2021-task1.zip'
fi

In [4]:
%%bash
path='/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats'
dst='/content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled/'

rm -r '/content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled'
if [ ! -d '/content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled' ]; then

    mkdir '/content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled'

    rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/train
    rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/test
    rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/val

    for el in $(ls $path | head -n 3);
        do
            echo "$path/$el -> $dst"
            cp -R "$path/$el" $dst
        done
fi


/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats/BraTS2021_00000 -> /content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled/
/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats/BraTS2021_00002 -> /content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled/
/content/drive/MyDrive/Lorusso/BraTS/data/raw/brats/BraTS2021_00003 -> /content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled/


## MLFlow server

In [27]:
mlflow.set_tracking_uri('file:///content/drive/MyDrive/Lorusso/BraTS/mlruns')

# Terminate open tunnels if exist
ngrok.kill()
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2Yliuv8VnNyKNcljxgEv6NpZgz8_6ZDBYmEcebUeoX93eGJAE"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --backend-store-uri file:///content/drive/MyDrive/Lorusso/BraTS/mlruns --port 5000 & ")


MLflow Tracking UI: https://2dff-34-125-53-119.ngrok-free.app


## Utils 🛠

In [6]:
def untar_brats(tar_path, extract_path):
    tar = tarfile.open(tar_path)
    tar.extractall(extract_path)
    tar.close()



def plot_reconstruction(im_orig, im_rec, ax:int = 0, slice_index:int = 100):

    f, ax_array = plt.subplots(1,2, figsize=(10,10))
    ax_array[0].imshow(np.take(im_orig, indices = slice_index, axis = ax), cmap='gray')
    ax_array[1].imshow(np.take( im_rec , indices=slice_index, axis = ax), cmap='gray')

def plot_brain_sections(images,ax = 1,slice_index = 90):

    """
    """

    d1,d2,d3 = np.shape(images[1])
    dims = [d1,d2,d3]
    dims.pop(ax)
    color_segmentation = np.zeros((dims[0],dims[1],3))

    gray_segmentation = np.take(images[1],slice_index,axis = ax)
    color_segmentation[gray_segmentation == 1] = [255,0,0] # Red (necrotic tumor core)
    color_segmentation[gray_segmentation == 2] = [0,255,0] # Green (peritumoral edematous/invaded tissue)
    color_segmentation[gray_segmentation == 4] = [0,0,255] # Blue (enhancing tumor)

    t1 = images[0][0]
    flair = images[0][1]
    t2 = images[0][2]
    t1ce = images[0][3]

    image = t1+t2+flair+t1ce

    plt.figure(figsize=(15,15))
    plt.subplot(1,4,1)
    plt.imshow(np.take(image,slice_index,axis = 0),cmap='gray')

    plt.subplot(1,4,2)
    plt.imshow(np.take(image,slice_index,axis = 1),cmap='gray')

    plt.subplot(1,4,3)
    plt.imshow(np.take(image,slice_index,axis = 2),cmap='gray')

    plt.subplot(1,4,4)
    plt.imshow(color_segmentation,cmap='gray')
    plt.xlabel('Segmentation')


def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1):
    n,c,w,h = tensor.shape
    if allkernels: tensor = tensor.view(n*c, -1, w, h)
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)
    rows = np.min((tensor.shape[0] // nrow + 1, 64))
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure( figsize=(nrow,rows) )
    plt.imshow(grid.cpu().permute((1, 2, 0)))





## Dataset class  💾


In [7]:
'''
Preprocessing script to convert from data provided by BraTS to data used by our model. Should be the first thing you run.
Fulfills the following four functions:
1. Normalize and standardize each image of each MRI modality
2. Combine multiple MRI modalitities into one image array
3. Swap labels from BraTS order (0,2,1,4) to more intuitive order (0,1,2,3)
4. Convert image into a graph
    Using Simple Linear Iterative Clustering algorithm
    Parameters passed on command line

If no labels are present (e.g. at test time, in deployment) can also build graph without labels.

Saves the following in the specified output directory for each sample
MRI_ID/
    _input.nii.gz (processed and combined modalities for a sample as nifti file)
    _label.nii.gz
    _nxgraph.json (networkx graph containing both graph topography and features and labels for each node)
    _supervoxels.nii.gz (supervoxel partitioning produced by SLIC)
    _crop.npy (optionally the crop of the processed data relative to the original data) (crops out empty image planes)
'''


class DataPreprocessor(Dataset):
    def __init__(self, dotenv_path = "/content/drive/MyDrive/Lorusso/BraTS/.env", transform = True, INPUT_PATH = None,
                 num_nodes = 5000, boxiness_coef = 0.5, num_neighbors = 10):

        load_dotenv(dotenv_path)
        # Data mean and variance
        data_stats = ([0.4645, 0.6625, 0.4064, 0.3648],
                      [0.1593, 0.1703, 0.1216, 0.1627])
        self.N_THREADS = 6

        self.num_nodes = num_nodes
        self.boxiness_coef = boxiness_coef
        self.num_neighbors = num_neighbors

        if(INPUT_PATH is not None and os.path.exists(INPUT_PATH)):
            self.data_dir = INPUT_PATH
        else:
            self.data_dir = os.getenv('INPUT_DATA_DIR')
        self.output_dir = os.getenv('PROCESSED_DATA_DIR')
        self.mri_prefix = 'BraTS2021'
        self.modality_extensions = ["_flair.nii.gz", "_t1.nii.gz", "_t1ce.nii.gz", "_t2.nii.gz"]
        self.label_extension = "_seg.nii.gz"
        self.include_labels = self.label_extension is not None
        self.all_ids, self.id_to_fp = self.get_all_mris_in_dataset()
        self.LABEL_MAP = {4: 3, 2: 1, 1: 2}
        self.dataset_mean = np.array(data_stats[0], dtype=np.float32)
        self.dataset_std = np.array(data_stats[1], dtype=np.float32)
        self.transform = transform if isinstance(transform, bool) else True


    def __len__(self):
        return len(self.all_ids)


    def __getitem__(self,idx):
        images = []
        # Load the image corresponding to idx
        try:
            fp = [self.id_to_fp[k] for k in self.all_ids if k.split('_')[-1] == idx][0]
            bn = os.path.basename(os.path.split(fp)[0])
            images.append([nib.load(os.path.join(fp, bn + level)).get_fdata(dtype=np.float32).T
                           for level in self.modality_extensions])
            labels = nib.load(os.path.join(fp, bn + self.label_extension)).get_fdata(dtype=np.float32).T

            # Convert to numpy array otherwise you'll experience RAM leak
            images = np.asarray(images[0])
            imstack = np.stack(np.array(images, dtype=np.float32), axis = 0)
            imstack, labels = self.padding(imstack, labels)

            if (self.transform):
                imstack,labels = self.get_standardized_image(imstack, labels)

        except Exception as e:
            print(f"Exception thrown in class {self.__class__.__name__ }, method __getitem__")
            print(e)
            imstack = np.zeros([4,240,240,240],dtype=np.float32)
            labels = np.zeros([240, 240, 240])

        return np.array(imstack), labels

    def image_to_graph(self, mri_id):
        # NB: be careful to pass only the number to __getitem__ and not all the string.
        # Example: Given the MRI Brats_00000 we need to pass to image_to_graph only '00000'
        imstack, labels = self.__getitem__(mri_id.split('_')[-1])
        save_path = f"{self.output_dir}_{self.num_nodes}_{self.boxiness_coef}_{self.num_neighbors}{os.sep}{os.path.basename(self.data_dir)}{os.sep}{mri_id}"

        if not os.path.exists(save_path):
            os.makedirs(save_path)
            # TRANSPOSE variable imstack because img2graph expects channel-first images
            nx_graph,node_feats,region_img = img2graph(imstack.T,labels,self.num_nodes,self.boxiness_coef,self.num_neighbors)
            #save in correct folder

            save_networkx_graph(nx_graph, f"{save_path}{os.sep}{mri_id}_nxgraph.json")
            save_as_nifti(imstack,f"{save_path}{os.sep}{mri_id}_input.nii.gz")
            save_as_nifti(labels,f"{save_path}{os.sep}{mri_id}_label.nii.gz")
            save_as_nifti(region_img,f"{save_path}{os.sep}{mri_id}_supervoxels.nii.gz")

        return mri_id

    def run(self):
        with concurrent.futures.ThreadPoolExecutor(max_workers=self.N_THREADS) as executor:
            futures = [executor.submit(self.image_to_graph, mri_id) for mri_id in self.all_ids]
            print("Set up Threads, starting execution")
            for future in concurrent.futures.as_completed(futures):
                try:
                    mri_id = future.result()
                except Exception as exc:
                    print("Execption caught in DataPreprocessor.run")
                    print(f"{exc}")
                else:
                    print("Finished "+ mri_id)



    def get_all_mris_in_dataset(self):
        mri_folders = glob.glob(f"{self.data_dir}**/{self.mri_prefix}*/",
                                recursive=True)
        mri_folders = self.remove_incomplete_mris(mri_folders)
        scan_dic = {os.path.split(fp)[0].split("/")[-1]: fp for fp in mri_folders}
        if(len(mri_folders) == 0):
            print("No MRI found at " + self.data_dir)
        return list(scan_dic.keys()), scan_dic


    def remove_incomplete_mris(self, mri_folders):
        # if there are any you want to ignore just add them to this list
        removed_mris = []
        return [fp for fp in mri_folders if fp.split("/")[-2] not in removed_mris]


    def split_dataset(self, ratio = (.6,.2,.2),seed = 42):

        random.seed(seed)
        if(np.sum(ratio) != 1 or ratio is None):
          print("Error: ratio does not sum up to one.\nSwitching to default (.6,.2,.2))")
          ratio = (.6,.2,.2)

        train_length = int(len(self.all_ids)*ratio[0])
        val_length = int(len(self.all_ids)*ratio[1])
        test_length = int(len(self.all_ids)*ratio[2])
        pos = random.sample(range(0,len(self.all_ids)), len(self.all_ids))

        split_dict = {
            'train': [self.all_ids[i] for i in pos[:train_length]],
            'val': [self.all_ids[i] for i in pos[train_length :train_length + val_length]],
            'test': [self.all_ids[i] for i in pos[train_length + val_length:]]
        }

        for k in split_dict.keys():
          parent = '/'.join(self.data_dir.split('/')[:-1])
          dst = os.path.join(parent,k)

          try:
            # create train,val,test dirs
            if(not os.path.exists(dst)):
              os.mkdir(dst)

            # copy splitted data inside folders
            for id in split_dict[k]:
              if(not os.path.exists(os.path.join(dst,id))):
                 os.mkdir(os.path.join(dst,id))
              copy_tree(self.id_to_fp[id],os.path.join(dst,id))

          except Exception as e:
            print(f"Exception thrown in class {self.__class__.__name__ }, method split_dataset")
            print(e)


    def padding(self,image, labels):
        n_channels = np.shape(image)[0]
        max_val = max(np.shape(image))
        pad_list = np.zeros([n_channels,max_val,max_val,max_val],dtype=np.float32)

        for channel in range(0, n_channels): # pad every channel
            pad_list[channel] = np.pad(image[channel],[(42,43),(0,0),(0,0)],'constant')
        labels = np.pad(labels, [(42,43),(0,0),(0,0)],'constant')

        return pad_list, labels


    def get_standardized_image(self, image_data, label_data):

        standardized_labels = self.swap_labels_from_brats(label_data)
        #standardized_data = self.standardize_img(
        #    image_data, self.dataset_mean, self.dataset_std)
        #normalized_data = self.standardize_img(image_data)
        normalized_data = self.normalize_img(image_data)
        return normalized_data, standardized_labels


    def normalize_img(self, img_array):
        new_image = np.zeros(img_array.shape, dtype=np.float32)
        n_channel = img_array.shape[0] # channel-first images

        for channel in range(0, n_channel): # normalize every channel

            maxval, minval= np.max(img_array[channel]), np.min(img_array[channel])
            new_image[channel] = (img_array[channel] - minval)/(maxval-minval)

        return new_image




    def normalize_img_quantile(self, img_array):
        quantile = np.quantile(img_array, 0.995, axis = (0,1,2,3) )
        return img_array/quantile



    def standardize_img(self,img_array):
        img_array = img_array.T # Align shapes
        centered = img_array-self.dataset_mean
        standardized = centered/self.dataset_std
        return standardized.T


    def swap_labels_from_brats(self,label_data):
        uniques = np.unique(label_data)
        for u in uniques:
            if u not in [0, 1, 2, 4]:
                raise RuntimeError('unexpected label')
        new_label_data = np.zeros_like(label_data, dtype=np.int16)
        new_label_data[label_data == 4] = self.LABEL_MAP[4]
        new_label_data[label_data == 2] = self.LABEL_MAP[2]
        new_label_data[label_data == 1] = self.LABEL_MAP[1]
        return new_label_data

    def swap_labels_to_brats(self,label_data):
        uniques = np.unique(label_data)
        for u in uniques:
            if u not in [0, 1, 2, 3]:
                raise RuntimeError('unexpected label')
        new_label_data = np.zeros_like(label_data, dtype=np.int16)
        new_label_data[label_data == self.LABEL_MAP[4]] = 4
        new_label_data[label_data == self.LABEL_MAP[2]] = 2
        new_label_data[label_data == self.LABEL_MAP[1]] = 1
        return new_label_data


'''
A Dataset similar to a torch dataset which iterates over all samples in a directory and returns the contents as numpy arrays.
Expects to receive a filepath to the output of the preprocess script which should have the following:
1.processed image (nifti)
2.label image (nifti)
3.networkx graph (json)
4.supervoxel partitioning (nifti)
5. (optionally) a .npy file containing the crop of the processed image relative to the original image


#Input#
dataset_root_dir: filepath to preprocessed dataset (generated by running preprocess script)
mri_start_string: a prefix that every image folder starts with (can be empty string)
read_image: whether to read in and return preprocessed images for each sample (only necessary for CNN model)
read_graph: whether to return graphs for each sample (for training GNN)
read_label: whether to read in labels. Will be returned in vector form (one label per node if )

#Output#

If graph:
Returns a DGL Graph, features for each node, and (optionally) labels for each node
If image:
Returns a numpy image array and (optionally) a numpy label array

'''

class ImageGraphDataset(Dataset):
    def __init__(self, dataset_root_dir,mri_start_string,read_image=True,read_graph=True,read_label=True):
        self.dataset_root_dir=dataset_root_dir
        self.all_ids = self.get_all_mris_in_dataset(dataset_root_dir,mri_start_string)
        self.read_image=read_image
        self.read_graph=read_graph
        self.read_label = read_label
        assert(self.read_graph or self.read_image)

    def get_all_mris_in_dataset(self,dataset_root_dir,mri_start_string):
        mri_folders = glob.glob(f"{dataset_root_dir}**/{mri_start_string}*/",recursive=True)
        mri_ids = [fp.split(os.sep)[-2] for fp in mri_folders]
        print(f"Found {len(mri_folders)} MRIs")
        return mri_ids

    def get_one(self,mri_id):
        if(self.read_graph and not self.read_image):
            return (mri_id, *self.get_graph(mri_id))
        elif(self.read_image  and not self.read_graph):
            return (mri_id, *self.get_image(mri_id))
        elif(self.read_image and self.read_graph):
            return (mri_id, *self.get_graph(mri_id), *self.get_image(mri_id))
        else:
            print("Invalid combination of flags")

    '''
    Reads in the saved networkx graph, converts it to a DGLGraph, normalizes the graph (not actually sure how useful this is),
    and returns the DGLGraph as well as a vector of node features and optionally labels.
    '''
    def get_graph(self,mri_id):
        nx_graph = load_networkx_graph(f"{self.dataset_root_dir}{os.sep}{mri_id}{os.sep}{mri_id}_nxgraph.json")
        features = np.array([nx_graph.nodes[n]['features'] for n in nx_graph.nodes])
        if(self.read_label):
            labels = np.array([nx_graph.nodes[n]['label'] for n in nx_graph.nodes])
        G = to_dgl_graph(nx_graph)
        n_edges = G.number_of_edges()
        # normalization
        degs = G.in_degrees().float()
        norm = torch.pow(degs, -0.5)
        norm[torch.isinf(norm)] = 0
        G.ndata['norm'] = norm.unsqueeze(1)
        #G.ndata['feat'] = features
        if(self.read_label):
            #G.ndata['label'] = labels
            return G, features, labels
        return G, features

    def get_voxel_labels(self,mri_id):
        fp=f"{self.dataset_root_dir}{os.sep}{mri_id}{os.sep}{mri_id}_label.nii.gz"
        return read_nifti(fp,np.int16)

    def get_image(self,mri_id):
        fp = f"{self.dataset_root_dir}{os.sep}{mri_id}{os.sep}{mri_id}_input.nii.gz"
        img = read_nifti(fp,np.float32)
        if(self.read_label):
            return img,self.get_voxel_labels(mri_id)
        else:
            return (img,)

    def get_supervoxel_partitioning(self,mri_id):
        fp=f"{self.dataset_root_dir}{os.sep}{mri_id}{os.sep}{mri_id}_supervoxels.nii.gz"
        return read_nifti(fp,np.int16)

    def get_crop(self,mri_id):
        fp=f"{self.dataset_root_dir}{os.sep}{mri_id}{os.sep}{mri_id}_crop.npy"
        return tuple(np.load(fp,allow_pickle=True))

    def __iter__(self):
        for mri_id in self.all_ids:
            yield self.get_one(mri_id)

    def __getitem__(self, index):
        mri_id = self.all_ids[index]
        return self.get_one(mri_id)

    def __len__(self):
        return len(self.all_ids)




class SeqSampler(SequentialSampler):
    """Samples elements sequentially, always in the same order.

    Args:
        data_source (Dataset): dataset to sample from
    """
    def __init__(self, data_source:Dataset):
        self.data_source = data_source
        self.indexDict = [id.split('_')[1] for id in data_source.all_ids]
    def __iter__(self):
        return iter(self.indexDict)
    def __len__(self):
        return len(self.indexDict)


## Models 📪


In [8]:

from dgl.nn.pytorch import GATConv, GraphConv
from dgl.nn.pytorch.conv import SAGEConv
import torch.nn as nn
import torch.nn.functional as F

'''
Contains the actual neural network architectures.
Supports GraphSAGE with either the pool,mean,gcn, or lstm aggregator as well as GAT.
The input, output, and intermediate layer sizes can all be specified.
Typically will call init_graph_net and pass along the desired model and hyperparameters.

Also contains the CNN Refinement net which is a very simple 2 layer 3D convolutional neural network.
As input, it expects 8 channels, which are the concatenated 4 input modalities and 4 output logits of the GNN predictions.
'''


class GraphSage(nn.Module):
    def __init__(self,in_feats,layer_sizes,n_classes,aggregator_type,dropout):
        super().__init__()
        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(SAGEConv(in_feats, layer_sizes[0], aggregator_type, feat_drop=dropout, activation=F.relu))
        # hidden layers
        for i in range(1,len(layer_sizes)):
            self.layers.append(SAGEConv(layer_sizes[i-1], layer_sizes[i], aggregator_type, feat_drop=dropout, activation=F.relu))
        # output layer
        self.layers.append(SAGEConv(layer_sizes[-1], n_classes, aggregator_type, feat_drop=0, activation=None))

    def forward(self,graph,features):
        h = features
        for layer in self.layers:
            h = layer(graph, h)
        return h


class GAT(nn.Module):
    def __init__(self,in_feats,layer_sizes,n_classes,heads,residuals,
                activation=F.elu,feat_drop=0,attn_drop=0,negative_slope=0.2):
        super().__init__()
        self.layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.layers.append(GATConv(
            in_feats, layer_sizes[0], heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))
        # hidden layers
        for i in range(1, len(layer_sizes)):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.layers.append(GATConv(
                layer_sizes[i-1] * heads[i-1], layer_sizes[i], heads[i],
                feat_drop, attn_drop, negative_slope, residuals[i], self.activation))
        # output projection
        self.layers.append(GATConv(
            layer_sizes[-1] * heads[-1], n_classes, 1,
            feat_drop, attn_drop, negative_slope, False, None))

    def forward(self,g, inputs):
        h = inputs
        for l in range(len(self.layers)-1):
            h = self.layers[l](g, h).flatten(1)
        # output projection
        logits = self.layers[-1](g, h).mean(1)
        return logits


## Model Wrapper 📨


In [9]:
class ModelWrapper():
    """
    Allows train, evaluation, prediction and I/O operations on generic PyTorch models
    The model is saved at every epoch.
    """

    def __init__(self, model: nn.Module, optimizer: nn.Module, loss_fn: nn.Module,
                 num_epochs: int, supervised: bool = True, dict_params:dict = {}, LOAD_MODEL: bool = False,
                 model_path: str  = '/content/drive/MyDrive/Lorusso/models', eval_metrics = None):

        self.device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
        self.model = model
        self.model = self.model.to(self.device)
        self.model = self.model.to(torch.float)
        self.num_epochs = num_epochs
        self.loss_fn = loss_fn
        self.eval_metrics = eval_metrics
        self.optimizer = optimizer
        self.model_path = model_path
        self.save_path = self.model_path + '/' + self.model.__class__.__name__ + '/model.pt'

        self.supervised = supervised
        self.training_loss = []
        self.validation_loss = []
        self.dict_metrics = {}
        self.elapsed_epochs = 0
        self.elapsed_seconds = 0

        self.dict_params = dict_params
        self.update_params({'loss_fn':self.loss_fn.__class__.__name__})
        self.update_params({'optimizer':self.optimizer.__class__.__name__})
        self.update_params({'learning_rate':self.optimizer.state_dict()['param_groups'][0]['lr']})
        self.update_params({'weight_decay':self.optimizer.state_dict()['param_groups'][0]['weight_decay']})

        if(LOAD_MODEL):
          self.load_checkpoint()

        # Create directory for model loading
        try:
          if(not os.path.exists(self.model_path + '/' + self.model.__class__.__name__)):
            os.mkdir(self.model_path + '/' + self.model.__class__.__name__)
        except Exception as e:
          print(f"Exception thrown in class {self.model.__class__.__name__ }, method __init__")
          print(e)
          print('\n')


    def update_params(self, new_params):
        try:
            self.dict_params.update(new_params)
        except Exception as e:
            print('Exception raised in WrapperModel.update_params')
            print(e)

    def log_checkpoint(self, info: dict):
        mlflow.pytorch.log_state_dict(info, artifact_path='checkpoint')
        #torch.save(info, self.save_path)

    def load_checkpoint(self):
        """ Loads the last checkpoint for the given model """
        try:
            run_id = mlflow.search_runs(experiment_names=['BraTS_'+type(self.model).__name__],
                                        order_by=["start_time DESC"]).iloc[0].run_id

            checkpoint = mlflow.pytorch.load_state_dict('runs:/'+run_id+'/checkpoint',
                                                        map_location=torch.device(self.device))

            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.elapsed_epochs = len(checkpoint['training_loss'])
            self.training_loss = checkpoint['training_loss']
            self.validation_loss = checkpoint['validation_loss']
            self.dict_metrics = checkpoint['dict_metrics']
            self.elapsed_seconds = checkpoint['elapsed_seconds']


        except Exception as e:
            print(f"Exception thrown in class {self.model.__class__.__name__ }, method load_checkpoint")
            print(e)
            print('\n')
            if(mlflow.active_run()):
                mlflow.end_run()


    def train(self, train_loader, val_loader = None,  experiment_prefix = ''):

        try:
            # Set MLFlow experiment
            if(experiment_prefix):
                mlflow.set_experiment(experiment_prefix + self.model.__class__.__name__)
            else:
                mlflow.set_experiment('BraTS_'+self.model.__class__.__name__)

            # Start a new run if the model wasn't loaded
            if(not mlflow.active_run()):
                # Track metrics in the current run
                mlflow.start_run()

            for i in range(len(self.training_loss)):
                mlflow.log_metric('train_loss', self.training_loss[i], step=i)

            for i in range(len(self.validation_loss)):
                mlflow.log_metric('val_loss', self.validation_loss[i], step=i)

            self.update_params({'batch_size':train_loader.batch_size})
            mlflow.log_params(self.dict_params)

            training_loss = self.training_loss
            validation_loss = self.validation_loss

            self.tot_epochs = self.elapsed_epochs + self.num_epochs+1
            self.tot_time = time.time()

            # Train
            for epoch in range(self.elapsed_epochs+1, self.tot_epochs):
                start = time.time() # track time

              # Evaluate first if loaded model missed the evaluation during an epoch
                if(len(training_loss) > len(validation_loss)):
                    if(val_loader is not None):

                      ## COMPLETE EVALUATION OF PREVIOUS EPOCH ##
                      # NB: epoch = epoch - 1
                        val_batch_loss = self.__eval(val_loader, (epoch-1) )
                        validation_loss.append(np.array(val_batch_loss).mean())
                        self.dict_metrics = self.__eval_metrics(val_loader)

                        for k in self.dict_metrics.keys():
                            mlflow.log_metric(str(k), self.dict_metrics[k], step=epoch-1)
                        # Log metric
                        mlflow.log_metric('val_loss',validation_loss[-1], step=epoch-1)

                        print(f"Epoch: {epoch-1}/{self.tot_epochs-1}, Validation Loss: {validation_loss[-1]:.4f}, Elapsed time: {time.time() - start:.0f} sec ")

                      # Update training time
                        epoch_time = int(time.time() - self.tot_time) + self.elapsed_seconds

                      #Create checkpoint
                        val_dict = {
                                  'model_state_dict': self.model.state_dict(),
                                  'optimizer_state_dict': self.optimizer.state_dict(),
                                  'training_loss': training_loss,
                                  'validation_loss': validation_loss,
                                  'dict_metrics': self.dict_metrics,
                                  'elapsed_seconds': epoch_time
                                  }
                        self.log_checkpoint(val_dict)
                    else:
                        # Kind of exception, needed to keep the vectors of the same size
                        validation_loss.append(np.mean(validation_loss))

                        #Log metric
                        mlflow.log_metric('val_loss',validation_loss[-1], step=epoch-1)

                #### TRAIN ######
                train_batch_loss = self.__train(train_loader, epoch)
                training_loss.append(np.array(train_batch_loss).mean())

                # Log metric
                mlflow.log_metric('train_loss',training_loss[-1], step=epoch) # MLFLOW tracking
                print(f"\nEpoch: {epoch}/{self.tot_epochs-1}, Loss: {training_loss[-1]:.4f}, Epoch elapsed time: {time.time() - start:.0f} sec \n")

                #Save model every elapsed epoch
                self.elapsed_epochs = epoch
                epoch_time = int(time.time() - self.tot_time) + self.elapsed_seconds

                train_dict = {
                          'model_state_dict': self.model.state_dict(),
                          'optimizer_state_dict': self.optimizer.state_dict(),
                          'training_loss': training_loss,
                          'validation_loss': validation_loss,
                          'dict_metrics':self.dict_metrics,
                          'elapsed_seconds': epoch_time
                          }
                self.log_checkpoint(train_dict)

                #### EVALUATE ######
                torch.cuda.empty_cache()
                time.sleep(5)
                if(val_loader is not None):

                    val_batch_loss = self.__eval(val_loader, epoch)
                    validation_loss.append(np.array(val_batch_loss).mean())

                    self.dict_metrics = self.__eval_metrics(val_loader)

                    for k in self.dict_metrics.keys():
                        mlflow.log_metric(str(k), self.dict_metrics[k], step=epoch)

                    # Log metric
                    mlflow.log_metric('val_loss',validation_loss[-1], step=epoch)
                    print(f"Epoch: {epoch}/{self.tot_epochs-1}, Validation Loss: {validation_loss[-1]:.4f}, Elapsed time: {time.time() - start:.0f} sec ")

                    #Checkpoint
                    epoch_time = int(time.time() - self.tot_time) + self.elapsed_seconds
                    val_dict = {
                              'model_state_dict': self.model.state_dict(),
                              'optimizer_state_dict': self.optimizer.state_dict(),
                              'training_loss': training_loss,
                              'validation_loss': validation_loss,
                              'dict_metrics':self.dict_metrics,
                              'elapsed_seconds': epoch_time
                              }
                    self.log_checkpoint(val_dict)

                print(f"Total training time: {time.time()-self.tot_time:.0f} sec")

            # Log model --> end run
            mlflow.pytorch.log_model(self.model, artifact_path='model')
            mlflow.end_run()
        except Exception as e:
            print(f"Exception thrown in class {self.model.__class__.__name__ }, method train:")
            print(e)
            print('\n')
            mlflow.end_run()


        return training_loss, validation_loss

    def __train(self, train_loader: DataLoader, epoch:int):

        """ Train for an epoch """

        self.model.train()
        train_batch_loss = []
        train_steps = int(len(train_loader.dataset.all_ids)/train_loader.batch_size)

        for i, (data,labels) in enumerate(train_loader):

          torch.cuda.empty_cache()
          self.optimizer.zero_grad()

          if self.device == 'cuda':
            data = data.type(torch.cuda.FloatTensor)
          else:
            data = data.type(torch.FloatTensor)

          data = data.to(self.device)

          outputs = self.model(data)

          if(self.supervised):
            loss = self.loss_fn(outputs,labels)
          else:
            loss = self.loss_fn(outputs, data)

          loss.backward()
          self.optimizer.step()
          train_batch_loss.append(loss.detach().item())

          out = f"Epoch: {epoch}/{self.tot_epochs-1}, Step: {i+1}/{train_steps}, Loss: {loss.item():.4f}, Elapsed time: {time.time() - self.tot_time:.0f} sec "
          sys.stdout.write("\r" + out)
          sys.stdout.flush()

        return train_batch_loss


    def __eval(self, val_loader: DataLoader, epoch:int):

        """ Evaluate for an epoch """

        val_steps = int(len(val_loader.dataset.all_ids)/val_loader.batch_size)
        self.model.eval()
        val_batch_loss = []

        with torch.no_grad():
          for i, (data,labels) in enumerate(val_loader):

            torch.cuda.empty_cache()
            self.optimizer.zero_grad()

            if self.device == 'cuda':
              data = data.type(torch.cuda.FloatTensor)
            else:
              data = data.type(torch.FloatTensor)

            data = data.to(self.device)
            outputs = self.model(data)

            if(self.supervised):
                val_loss = self.loss_fn(outputs, labels)
            else:
                val_loss = self.loss_fn(outputs, data)

            val_batch_loss.append(val_loss.detach().item())

            out = f"Epoch: {epoch}/{self.tot_epochs-1}, Validation Step: {i+1}/{val_steps}, Validation Loss: {val_loss.item():.4f}, Elapsed time: {time.time() - self.tot_time:.0f} sec "
            sys.stdout.write("\r" + out)
            sys.stdout.flush()

        return val_batch_loss


    def __eval_metrics(self, data_loader:DataLoader):
        """ Evaluates additional metrics aside the loss function """
        metrics_dict = {}
        try:
            self.model.eval()
            if(self.eval_metrics is not None):

                metrics_dict = {k.__class__.__name__:[] for k in self.eval_metrics}

                with torch.no_grad():

                    for i, (data,labels) in enumerate(data_loader):
                        torch.cuda.empty_cache()
                        self.optimizer.zero_grad()

                        for metric in self.eval_metrics:

                            if self.device == 'cuda':
                                data = data.type(torch.cuda.FloatTensor)
                            else:
                                data = data.type(torch.FloatTensor)

                            data = data.to(self.device)
                            outputs = self.model(data)

                            if(self.supervised):
                                metric_value = metric(outputs, labels)
                            else:
                                metric_value = metric(outputs, data)

                            metrics_dict[metric.__class__.__name__].append(metric_value.detach().item())

                    for k in metrics_dict.keys():
                        metrics_dict[k] = np.array(metrics_dict[k]).mean()

        except Exception as e:
            print('Exception thrown in wrapper.__eval_metrics')
            print(e)
        finally:
            return metrics_dict



    def predict_batch(self, data_loader:DataLoader):

        output = []
        self.model = self.model.to(self.device)
        self.model.eval()
        with torch.no_grad():
            try:
              for i, batch in enumerate(data_loader):
                torch.cuda.empty_cache()

                if(len(batch) == 1):
                  data = batch
                else:
                  data,label = batch

                if self.device == 'cuda':
                  data = data.type(torch.cuda.FloatTensor)
                else:
                  data = data.type(torch.FloatTensor)

                data.to(self.device)

                out = self.model(data)
                out = out.cpu().detach().numpy()
                output.append(out)

            except Exception as e:
                print(f"Exception thrown in class {self.model.__class__.__name__ }, method predict_batch:")
                print(e)
                print('\n')

        return np.array(output)


    def predict(self, data):

        if device == 'cuda':
          data = data.type(torch.cuda.FloatTensor)
        else:
          data = data.type(torch.FloatTensor)

        self.model.eval()
        with torch.no_grad():
            data.to(device)
            output = self.model(data)
        return output






## Loss 🕳

In [10]:
class Loss(nn.Module):
    def __init__(self, focal):
        super(Loss, self).__init__()
        if focal:
            self.loss_fn = DiceFocalLoss(
                include_background=False, softmax=True, to_onehot_y=True, batch=True, gamma=2.0
            )
        else:
            self.loss_fn = DiceCELoss(include_background=False, softmax=True, to_onehot_y=True, batch=True)

    def forward(self, y_pred, y_true):
        return self.loss_fn(y_pred, y_true)


class LossBraTS(nn.Module):
    def __init__(self, focal):
        super(LossBraTS, self).__init__()
        self.dice = DiceLoss(sigmoid=True, batch=True)
        self.ce = FocalLoss(gamma=2.0, to_onehot_y=False) if focal else nn.BCEWithLogitsLoss()

    def _loss(self, p, y):
        print('p '+str(p.size()))
        print('y '+str(y.size()))
        return self.dice(p, y) + self.ce(p, y.float())

    def forward(self, p, y):
        y_wt, y_tc, y_et = y > 0, ((y == 1) + (y == 3)) > 0, y == 3
        p_wt, p_tc, p_et = p[:, 0].unsqueeze(1), p[:, 1].unsqueeze(1), p[:, 2].unsqueeze(1)
        l_wt, l_tc, l_et = self._loss(p_wt, y_wt), self._loss(p_tc, y_tc), self._loss(p_et, y_et)
        return l_wt + l_tc + l_et

## Build dataset 🏗

In [11]:
dotenv_path = "/content/drive/MyDrive/Lorusso/BraTS/.env"
load_dotenv(dotenv_path)


INPUT_PATH = os.getenv("INPUT_DATA_DIR")
PROCESSED_PATH = os.getenv('PROCESSED_DATA_DIR')
INPUT_PATH_PARENT = '/'.join(INPUT_PATH.split('/')[:-1])

TRAIN_PATH = os.path.join(INPUT_PATH_PARENT,'train')
VAL_PATH = os.path.join(INPUT_PATH_PARENT,'val')
TEST_PATH = os.path.join(INPUT_PATH_PARENT,'test')

TAR_PATH = os.path.join(INPUT_PATH_PARENT,'BraTS2021_Training_Data.tar')
BUILD_DATASET = False

if(BUILD_DATASET):
  untar_brats(tar_path = '/content/drive/MyDrive/Lorusso/BraTS/data/raw/BraTS2021_Training_Data.tar', extract_path = INPUT_PATH )
  dataset = DataPreprocessor()
  dataset.split_dataset()

#dataset = DataPreprocessor()
#train_loader = DataLoader(dataset, sampler = SeqSampler(dataset), batch_size = 1, num_workers = 0)
#images, labels= next(iter(train_loader))
#plot_brain_sections([images[0], labels[0]])
#del images, labels, train_loader, dataset

## Train and predict ⌛

In [12]:
TRAIN_MODEL = False
LOAD_MODEL = False

torch.cuda.empty_cache()
time.sleep(10)
if(TRAIN_MODEL or LOAD_MODEL):
  num_workers = 0
  batch_size = 1
  num_epochs = 1
  lr = 0.005
  supervised = False

  # MONAI AUTOENCODER
  model = AutoEncoder(
         spatial_dims=3,
         kernel_size = 3,
         up_kernel_size = 3,
         in_channels=4,
         out_channels=4,
         channels=(5,),
         strides=(2,),
         inter_channels=(8, 16, 32),
         inter_dilations=(1, 2, 4),
         num_inter_units=2
     )

  optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=1e-10)
  loss_fn = nn.MSELoss() #SSIMLoss(spatial_dims=3)

  wrapper = ModelWrapper(model = model,
                            loss_fn = loss_fn,
                            optimizer = optimizer,
                            supervised = supervised,
                            num_epochs = num_epochs,
                            LOAD_MODEL = LOAD_MODEL
                        )

  dataset = DataPreprocessor(INPUT_PATH = INPUT_PATH)

  # Split dataset if it's not
  if(not os.path.exists(TRAIN_PATH)):
    dataset.split_dataset()


  train_dataset = DataPreprocessor(INPUT_PATH = TRAIN_PATH)
  val_dataset = DataPreprocessor(INPUT_PATH = VAL_PATH)
  test_dataset = DataPreprocessor(INPUT_PATH = TEST_PATH)

  train_loader = DataLoader(dataset = train_dataset,
                           sampler = SeqSampler(train_dataset),
                           batch_size = batch_size,
                           num_workers = num_workers)

  val_loader = DataLoader(dataset = val_dataset,
                           sampler = SeqSampler(val_dataset),
                           batch_size = batch_size,
                           num_workers = num_workers)

  test_loader = DataLoader(dataset = test_dataset,
                           sampler = SeqSampler(test_dataset),
                           batch_size = batch_size,
                           num_workers = num_workers)


  print('Elapsed epochs: ' + str(wrapper.elapsed_epochs))

if(TRAIN_MODEL):
      training_loss, validation_loss = wrapper.train(train_loader = train_loader, val_loader = val_loader )
      torch.cuda.empty_cache()

In [13]:
if(TRAIN_MODEL or LOAD_MODEL):
  xx = range(1,wrapper.elapsed_epochs+1)
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.plot(xx, wrapper.training_loss, '-o', label = 'Train')
  plt.plot(xx, wrapper.validation_loss,'-o', label = 'Val')
  plt.legend(loc='lower left')

In [14]:
if(TRAIN_MODEL or LOAD_MODEL):

  slice_index = 150
  ax = 0

  im_test, lab = next(iter(test_loader))
  output = wrapper.predict(data = im_test)
  out = output[0]
  out_numpy = out.cpu().detach().numpy()
  im_test_numpy = im_test.cpu().detach().numpy()

  plot_reconstruction(im_orig = np.sum(im_test_numpy[0], axis=0), im_rec = np.sum(out_numpy, axis = 0), ax = ax, slice_index = slice_index)
  plot_brain_sections([im_test[0], lab[0]], ax = ax, slice_index = slice_index)

In [15]:
if(TRAIN_MODEL or LOAD_MODEL):
  ker = wrapper.model.decode[0][0][0].weight.detach().clone()
  print(ker.size())
  #
  visTensor(ker[0], allkernels=True)

  ker = wrapper.model.decode[0][0][0].weight.detach().clone()
  print(ker.size())
  #
  visTensor(torch.sum(ker, dim=(0)), allkernels=True)

  kk = torch.sum(ker, dim= 0)
  kk = torch.sum(kk, dim = 0)
  plt.imshow(kk[0,:,:].cpu().detach().numpy())

# TESTING SECTION 🚧

In [16]:
#! rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled
#! rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/train
#! rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/test
#! rm -r /content/drive/MyDrive/Lorusso/BraTS/data/interim/val

## Autoencoder

In [17]:

TEST_MODE = False

torch.cuda.empty_cache()
if(TEST_MODE):

    INPUT_PATH = '/content/drive/MyDrive/Lorusso/BraTS/data/interim/sampled'
    INPUT_PATH_PARENT = '/'.join(INPUT_PATH.split('/')[:-1])
    TRAIN_PATH = os.path.join(INPUT_PATH_PARENT,'train')
    VAL_PATH = os.path.join(INPUT_PATH_PARENT,'val')
    TEST_PATH = os.path.join(INPUT_PATH_PARENT,'test')

    TRAIN_MODEL = False
    LOAD_MODEL = False # resume training

    num_workers = 0
    batch_size = 1
    num_epochs = 2
    lr = 0.01
    supervised = False
    eval_metrics = [nn.MSELoss()]

  # MONAI AUTOENCODER
    model = AutoEncoder(
           spatial_dims=3,
           kernel_size = 3,
           up_kernel_size = 3,
           in_channels=4,
           out_channels=4,
           channels=(5,),
           strides=(2,),
           inter_channels=(8, 16, 32),
           inter_dilations=(1, 2, 4),
           num_inter_units=2
       )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=1e-10)
    loss_fn = nn.MSELoss() #SSIMLoss(spatial_dims=3)

    wrapper = ModelWrapper(model = model,
                              loss_fn = loss_fn,
                              optimizer = optimizer,
                              supervised = supervised,
                              num_epochs = num_epochs,
                              LOAD_MODEL = LOAD_MODEL,
                              eval_metrics = eval_metrics
                          )

    dataset = DataPreprocessor(INPUT_PATH = INPUT_PATH)

    # Split dataset if it's not
    if(not os.path.exists(TRAIN_PATH)):
      dataset.split_dataset()


    train_dataset = DataPreprocessor(INPUT_PATH = TRAIN_PATH)
    val_dataset = DataPreprocessor(INPUT_PATH = VAL_PATH)
    test_dataset = DataPreprocessor(INPUT_PATH = TEST_PATH)

    train_loader = DataLoader(dataset = train_dataset,
                             sampler = SeqSampler(train_dataset),
                             batch_size = batch_size,
                             num_workers = num_workers)

    val_loader = DataLoader(dataset = val_dataset,
                             sampler = SeqSampler(val_dataset),
                             batch_size = batch_size,
                             num_workers = num_workers)

    test_loader = DataLoader(dataset = test_dataset,
                             sampler = SeqSampler(test_dataset),
                             batch_size = batch_size,
                             num_workers = num_workers)


    print('Elapsed epochs: ' + str(wrapper.elapsed_epochs))

    if(TRAIN_MODEL):
        training_loss, validation_loss = wrapper.train(train_loader = train_loader, val_loader = val_loader, experiment_prefix = 'Test' )
        torch.cuda.empty_cache()


In [18]:
if(TRAIN_MODEL or LOAD_MODEL):
  xx = range(1,wrapper.elapsed_epochs+1)
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.plot(xx, wrapper.training_loss, '-o', label = 'Train')
  plt.plot(xx, wrapper.validation_loss,'-o', label = 'Val')
  plt.legend(loc='lower left')

In [19]:
if(TRAIN_MODEL or LOAD_MODEL):

  slice_index = 90
  ax = 0

  im_test, lab = next(iter(test_loader))
  output = wrapper.predict(data = im_test)
  out = output[0]
  out_numpy = out.cpu().detach().numpy()
  im_test_numpy = im_test.cpu().detach().numpy()

  plot_reconstruction(im_orig = np.sum(im_test_numpy[0], axis=0), im_rec = np.sum(out_numpy, axis = 0), ax = ax, slice_index = slice_index)
  plot_brain_sections([im_test[0], lab[0]], ax = ax, slice_index = slice_index)

In [20]:
if(TRAIN_MODEL or LOAD_MODEL):
  ker = wrapper.model.decode[0][0][0].weight.detach().clone()
  print(ker.size())
  #
  visTensor(ker[1], allkernels=True)

  ker = wrapper.model.decode[0][0][0].weight.detach().clone()
  print(ker.size())
  #
  visTensor(torch.sum(ker, dim=(0)), allkernels=True)

  kk = torch.sum(ker, dim= 0)
  kk = torch.sum(kk, dim = 0)
  plt.imshow(kk[0,:,:].cpu().detach().numpy())

## GNN training ⛽

In [21]:
#start = time.time()
## Create graphs
#train_dataset.run()
#end = time.time() - start
#end

In [26]:

TEST_MODE = True

torch.cuda.empty_cache()
if(TEST_MODE):

    INPUT_PATH = '/content/drive/MyDrive/Lorusso/BraTS/data/processed_5000_0.5_10'
    INPUT_PATH_PARENT = '/'.join(INPUT_PATH.split('/')[:-1])
    TRAIN_PATH = os.path.join(INPUT_PATH_PARENT,'train')
    VAL_PATH = os.path.join(INPUT_PATH_PARENT,'val')
    TEST_PATH = os.path.join(INPUT_PATH_PARENT,'test')

    TRAIN_MODEL = True
    LOAD_MODEL = False # resume training

    num_workers = 0
    batch_size = 1
    num_epochs = 2
    lr = 0.01
    supervised = True
    eval_metrics = []

    dropout = 0
    input_feats = 20
    class_weights = torch.Tensor([0.1,1,2,2])
    layer_sizes=[256]*4
    n_classes=4
    aggregator_type='pool'

    dict_params = {k:eval(k) for k in ['input_feats', 'class_weights', 'layer_sizes', 'n_classes', 'aggregator_type', 'n_classes']}

    model = GraphSage(in_feats=input_feats,layer_sizes=layer_sizes,n_classes=n_classes,aggregator_type=aggregator_type,dropout=dropout)

    optimizer = torch.optim.AdamW(model.parameters(),lr=lr,weight_decay=1e-10)
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

    wrapper = ModelWrapper(model = model,
                              loss_fn = loss_fn,
                              optimizer = optimizer,
                              supervised = supervised,
                              dict_params = dict_params,
                              num_epochs = num_epochs,
                              LOAD_MODEL = LOAD_MODEL,
                              eval_metrics = eval_metrics
                          )


    train_dataset = ImageGraphDataset(TRAIN_PATH,'BraTS2021',read_image=False,read_graph=True,read_label=True)
    val_dataset = ImageGraphDataset(VAL_PATH,'BraTS2021',read_image=False,read_graph=True,read_label=True)
    test_dataset = ImageGraphDataset(TEST_PATH,'BraTS2021',read_image=False,read_graph=True,read_label=True)

    train_loader = DataLoader(dataset = train_dataset,
                             sampler = SeqSampler(train_dataset),
                             batch_size = batch_size,
                             num_workers = num_workers)

    val_loader = DataLoader(dataset = val_dataset,
                             sampler = SeqSampler(val_dataset),
                             batch_size = batch_size,
                             num_workers = num_workers)

    test_loader = DataLoader(dataset = test_dataset,
                             sampler = SeqSampler(test_dataset),
                             batch_size = batch_size,
                             num_workers = num_workers)


    print('Elapsed epochs: ' + str(wrapper.elapsed_epochs))

    if(TRAIN_MODEL):
        training_loss, validation_loss = wrapper.train(train_loader = train_loader, val_loader = None, experiment_prefix = 'Test_GNN' )
        torch.cuda.empty_cache()


Found 0 MRIs
Found 0 MRIs
Found 0 MRIs
Elapsed epochs: 0


2023/12/13 18:32:20 INFO mlflow.tracking.fluent: Experiment with name 'Test_GNNGraphSage' does not exist. Creating a new experiment.
<ipython-input-9-0e7b9eac4fcb>:157: RuntimeWarning: Mean of empty slice.
  training_loss.append(np.array(train_batch_loss).mean())
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Epoch: 1/2, Loss: nan, Epoch elapsed time: 0 sec 

Total training time: 5 sec

Epoch: 2/2, Loss: nan, Epoch elapsed time: 0 sec 



/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
2023/12/13 18:32:30 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Total training time: 10 sec


2023/12/13 18:32:36 WARNING mlflow.utils.requirements_utils: Found torch version (2.1.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# MLFlow transitioner

Helps to bring a model saved with pickle to MLFlow

In [ ]:
if(False):
    checkpoint = torch.load(wrapper.save_path, map_location=torch.device(wrapper.device))
    wrapper.model.load_state_dict(checkpoint['model_state_dict'])
    wrapper.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    wrapper.elapsed_epochs = checkpoint['epochs']
    wrapper.training_loss = checkpoint['training_loss']
    wrapper.validation_loss = checkpoint['validation_loss']
    wrapper.elapsed_seconds = checkpoint['elapsed_seconds']
    try:
        mlflow.set_experiment('BraTS_'+wrapper.model.__class__.__name__)
        mlflow.start_run()
        param_dict = {
            'batch_size':train_loader.batch_size,
            'optimizer':wrapper.optimizer.__class__.__name__,
            'learning_rate':wrapper.optimizer.state_dict()['param_groups'][0]['lr'],
            'loss_fn':wrapper.loss_fn.__class__.__name__
        }
        mlflow.log_params(param_dict)

        #sample_input,_ = next(iter(train_loader))
        #sample_output = wrapper.predict(sample_input)
        #signature = infer_signature(sample_input.numpy(), sample_output.numpy())
    #
        #print("Model signature:", signature)

        for i in range(len(wrapper.training_loss)):
            mlflow.log_metric('train_loss',wrapper.training_loss[i], step =i+1)
        for i in range(len(wrapper.validation_loss)):
            mlflow.log_metric('val_loss',wrapper.validation_loss[i], step =i+1)

        val_dict = {
              'model_state_dict': wrapper.model.state_dict(),
              'optimizer_state_dict': wrapper.optimizer.state_dict(),
              'elapsed_seconds': wrapper.elapsed_seconds,
              'training_loss':wrapper.training_loss,
              'validation_loss':wrapper.validation_loss
              }

        mlflow.pytorch.log_state_dict(val_dict,artifact_path="checkpoint")
        mlflow.pytorch.log_model(wrapper.model, artifact_path='model')

        mlflow.end_run()
    except Exception as e:
        print(e)
        mlflow.end_run()